In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [2]:
from pathlib import Path

# input_path = Path("../lc-project-data")
# output_path = Path("../lc-project-data")

# for MP,
# input_path = Path("../Data/DataV3")
# output_path = Path("OUTPUT/MP/05-classifiers/DataV3/")

#DataV4 - 2024-10-09 - Dropped patients with missing vax data
input_path = Path("../Data/DataV4")
output_path = Path("OUTPUT/MP/05-classifiers/DataV4/")


In [3]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

lPrettyCols = [
    "MainDataset",
    "RunType",
    "classifier",
    "brt_nTrue",
    "brt_nTop",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
    "brt_md",
]

In [13]:
# lDataNames = ['T81','T85','DT']


lDataNames = [
    # "dfcmplt", # Done
    # "dfcmplt_SITE1", # Done
    # "dfcmplt_SITE2", # Done
    # "dfcmplt_SITE3", # Done
    # "dfcmplt_SITE4", # Done
    # "dfcmplt_SITE5",  # LEFT
    # "dfcmplt_SITE6",  # LEFT
    # "dfcmplt_SITE7",  # LEFT
    # "dfcmpltPreLC",  # Done
    # "dfcmpltPreLC_SITE1", # Done
    # "dfcmpltPreLC_SITE2", # Done
    # "dfcmpltPreLC_SITE3", # Done
    # "dfcmpltPreLC_SITE4", # Done
    # "dfcmpltPreLC_SITE5",  # Done
    # "dfcmpltPreLC_SITE6",  # Done
    # "dfcmpltPreLC_SITE7",  # Done
    # "dfcmpltPreLC2",  # Done
    # "dfcmpltPreLC2_SITE1", # Done
    # "dfcmpltPreLC2_SITE2", # Done
    # "dfcmpltPreLC2_SITE3", # Done
    # "dfcmpltPreLC2_SITE4", # Done
    # "dfcmpltPreLC2_SITE5",  # Done
    # "dfcmpltPreLC2_SITE6",  # Done
    # "dfcmpltPreLC2_SITE7",  # Done
    # "dfcmpltPreLC3",  # Done
    # "dfcmpltPreLC3_SITE1", # Done
    # "dfcmpltPreLC3_SITE2", # Done
    # "dfcmpltPreLC3_SITE3", # Done
    # "dfcmpltPreLC3_SITE4", # Done
    # "dfcmpltPreLC3_SITE5",  # Done
    # "dfcmpltPreLC3_SITE6",  # Done
    # "dfcmpltPreLC3_SITE7",  # Done
    # "dfcmpltPreLC4",  # Done
    # "dfcmpltPreLC4_SITE1", # Done
    # "dfcmpltPreLC4_SITE2", # Done
    # "dfcmpltPreLC4_SITE3", # Done
    "dfcmpltPreLC4_SITE4", # Done
    # "dfcmpltPreLC4_SITE5",  # 
    # "dfcmpltPreLC4_SITE6",  # 
    # "dfcmpltPreLC4_SITE7",  # 
]
dataDir = "../Data/DataV4/TTS/"

In [14]:
# saveDir = MPutils.get_saving_dir(f"OUTPUT/MP/05-classifiers/DataV2/{dataName}/")


lClassifiers = ["RF", "LogR",'XGBcpu'] # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
lCV = [5] # different cross-validation splits
lTrees=[100,1000] #num trees
lMD=[3,6,10,15, None] #max depth

lBrt_MD = [3, 5, 7]
brt_trees = 1000
brt_iter = 500
brt_thres = 100
brt_toprank = 5


# FIXME - BELOW CODE FOR TESTING PURPOSES
# dataName = 'dfcmplt'
# lClassifiers = [
#     "RF",
#     "LogR",
#     "XGBcpu",
# ]  # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
# lCV = [5]  # different cross-validation splits
# lTrees = [10]  # num trees
# lMD = [3]  # max depth

# lBrt_MD = [4]
# brt_trees=12
# brt_iter = 10
# brt_thres=100
# brt_toprank=5

# Automated ML BRT and testing

In [15]:
import os


def updateExistingDF(dfTmp, fileName, save=False):
    if os.path.exists(fileName):
        dfTmp = pd.concat([dfTmp, pd.read_csv(fileName)])
        if 'params' in dfTmp.columns:
            dfTmp["params"] = dfTmp["params"].iloc[0].__str__()
        dfTmp = dfTmp.drop_duplicates(
             subset=[x for x in dfTmp.columns if x != "param_clf" and x!='date' and 'split' not in x and '_time' not in x and 'rank_' not in x and 'std' not in x]
        )
    if save:
        dfTmp.to_csv(fileName, index=False)

    return dfTmp

In [ ]:
lRunDataFrames = []
for dataName in tqdm(lDataNames,desc='DataSet Main Outer Loop'):

    df_tr = load_dataset_to_df(f"{dataDir}{dataName}_Train.arrow", verbose=True)
    df_ts = load_dataset_to_df(f"{dataDir}{dataName}_Test.arrow", verbose=True)
    df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)
    df_full = pd.concat([df_tr,df_ts,df_h])

    lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']

    for df in [df_tr,df_ts,df_h,df_full]:
        df.drop(df[df["LC_STATUS"] == 2].index, inplace=True)  # drop HC
        for c in lColDrop:
            if c in df.columns:
                df.drop(columns=[c],inplace=True)
        ##Depreciated below based on doing this in the splice data
        # df["LC_STATUS"] = df["LC_STATUS"].apply(
        #     lambda x: 1 if x == "LC_POS" else 0
        # )  # Convert to 0==LCNeg, 1==LCPos
        # df.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

    saveDir = MPutils.get_saving_dir(
        f"OUTPUT/MP/05-classifiers/DataV4/{dataName}/"
    )

    ## HOLDOUT SET ONLY
    x_h, y_h = getXY(df_h)

    # Ensure all arguments are picklable -- due to BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
    picklable_lScorers = [scorer for scorer in lScorersBinary]
    picklable_lClassifiers = [classifier for classifier in lClassifiers]

    dfRHold = classify_MP(
        X=x_h,
        y=y_h,
        lScorers=picklable_lScorers,
        lClassifiers=picklable_lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
        n_jobs=-1
    )

    dfRHold["Title"] = f"{dataName}_Holdout"
    dfRHold['RunType']='Holdout'

    dfRHold["MainDataset"] = dataName
    dfRHold["date"] = datetime.today()

    holdFile = f"{saveDir}CA_{dataName}_Holdout.csv"
    dfRHold = updateExistingDF(dfRHold, holdFile, save=True)

    ## Full SET ONLY
    x_F, y_F = getXY(df_full)

    dfRFull = classify_MP(
        X=x_F,
        y=y_F,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
        n_jobs=-1,
    )
    dfRFull["Title"] = f"{dataName}_Full"
    dfRFull["RunType"] = "Full"
    dfRFull["MainDataset"] = dataName
    dfRFull["date"] = datetime.today()

    fulldataFile= f"{saveDir}CA_{dataName}_Full.csv"
    dfRFull = updateExistingDF(dfRFull, fulldataFile, save=True)

    ## SECTION BORUTA Feature Selection

    x_Tr, y_Tr = getXY(df_tr)
    lBrtResults = []
    for boruta_md in tqdm(lBrt_MD[:],desc='Boruta MD Loop'):
        # run brouta = get true and top feat
        trueFeat, topFeat = boruta_fs(
            X=x_Tr.values,
            y=y_Tr,
            feat_list=x_Tr.columns,
            trees=brt_trees,
            ittr=brt_iter,
            threshold=brt_thres,
            top_rank=brt_toprank,
            verbose=0,
            model=RandomForestClassifier(
                n_jobs=-1, class_weight="balanced", max_depth=boruta_md, random_state=42
            ),
            fileName=f"{saveDir}FS_{dataName}_Boruta_T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}_MD{boruta_md}.csv",
        )

        ## Test Brouta Features
        x_TsMain, y_Ts = getXY(df_ts)

        x_Ts_True = x_TsMain[trueFeat]
        x_Ts_Top = x_TsMain[topFeat]

        # SECTION - True Feat
        if len(trueFeat) > 0:
            dfR_True = classify_MP(
                X=x_Ts_True,
                y=y_Ts,
                lScorers=lScorersBinary,
                lClassifiers=lClassifiers,
                lTrees=lTrees,
                lMD=lMD,
                lCV=lCV,
                n_jobs=-1,
            )
            dfR_True["Title"] = f"{dataName}_Boruta_True"
            dfR_True["brt_md"] = boruta_md
            dfR_True['brt_nTrue'] = len(trueFeat)
            dfR_True['brt_nTop'] = len(topFeat)
            dfR_True["RunType"] = "Boruta_True"
            lBrtResults.append(dfR_True)

        # SECTION - Top Feat
        if len(topFeat) > 0:
            dfR_Top = classify_MP(
                X=x_Ts_Top,
                y=y_Ts,
                lScorers=lScorersBinary,
                lClassifiers=lClassifiers,
                lTrees=lTrees,
                lMD=lMD,
                lCV=lCV,
                n_jobs=-1,
            )
            dfR_Top["Title"] = f"{dataName}_Boruta_Top"

            dfR_Top["brt_md"] = boruta_md
            dfR_Top['brt_nTrue'] = len(trueFeat)
            dfR_Top["brt_nTop"] = len(topFeat)
            dfR_Top["RunType"] = "Boruta_Top"
            lBrtResults.append(dfR_Top)

    dfCLFRun = pd.concat([dfRHold, dfRFull]+lBrtResults)

    dfCLFRun["MainDataset"] = dataName
    dfCLFRun["date"] = datetime.today()
    dfCLFRun["brt_params"] = (
        f"T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}"
    )

    clfFile = f"{saveDir}CLFRun_{dataName}_Results.csv"

    dfCLFRun = updateExistingDF(dfCLFRun,clfFile,save=False)
    dfCLFRun = MPutils.reorder_columns(dfCLFRun, ["MainDataset", "Title", "date",'brt_nTrue','brt_nTop'])
    dfCLFRun.to_csv(clfFile, index=False)

    # lRunDataFrames.append(dfCLFRun)

DataSet Main Outer Loop:   0%|          | 0/2 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]